# 크롤러 환경설정

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json
import os, sys
import tweepy

sys.path.append(os.pardir)
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
key_path = 'C:/Users/dkdud/Untitled Folder/api_key.json'
with open(key_path,'r') as f:
    key = json.load(f)

# 개인정보 인증을 요청하는 Handler입니다.
auth = tweepy.OAuthHandler(key['CONSUMER_KEY'], key['CONSUMER_SECRET'])

# 인증 요청을 수행합니다.
auth.set_access_token(key['ACCESS_TOKEN_KEY'], key['ACCESS_TOKEN_SECRET'])

# twitter API를 사용하기 위한 준비입니다.
api = tweepy.API(auth,wait_on_rate_limit=True)

In [5]:
#인스타 게시물을 불러오기 위한 사전작업 단계
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='data.campuss'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'data.2021'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)

def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

    
def get_content(driver):
    
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    tag_list =[]
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content, tag_data]
    return data

In [6]:
def insta_crawling(driver,keyword, target):
    
    result_list =[]
    url = insta_searching(keyword)
    driver.implicitly_wait(5)
    driver.get(url)
    time.sleep(5)

    click_first(driver)

    for i in range(target):
        try:
            result_list.append(get_content(driver))
            next_page(driver)
        except:
            time.sleep(2)
            next_page(driver)
    df=pd.DataFrame(result_list)
    df.columns=['내용','해시태그']
    df['내용'] = df['내용'].apply(lambda x : clean_text(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : str(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : clean_text(x))
    df['내용']= df['내용'] + df['해시태그']
    df.drop(['해시태그'],axis=1,inplace=True)
    df = df.drop_duplicates(subset=['내용'])
    return df

In [7]:
#트위터 게시물 가져오는 단계
def twitter_crawling(keyword):
    columns = ['내용']
    df = pd.DataFrame(columns=columns)

    max_tweets = 1000
    searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]

    for tweet in searched_tweets:
        tweet_json = tweet._json
        tweet_text = tweet_json['text']
        row = [tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

    df['내용'] = df['내용'].apply(lambda x : clean_text(x))
    df = df.drop_duplicates(subset=['내용'])
    
    return df


In [8]:
keyword = '환경오염'
target = 200

In [9]:
insta_df = insta_crawling(driver,keyword, target)

In [10]:
twitter_df = twitter_crawling(keyword)

In [11]:
sns_df = pd.concat([twitter_df, insta_df],join='outer',ignore_index=True)

In [23]:
sns_df.to_csv('환경오염.csv',encoding='UTF-8-SIG')

In [18]:
text= []
for x in range(0,len(sns_df['내용'])):
    try:
        tp = sns_df['내용'][x]
        tp = tp.split('.')
    except:
        print("0")
    text.append(tp)

In [19]:
import itertools
text=list(itertools.chain(*text))

from eunjeon import Mecab
tokenizer = Mecab()
pos_temp = {}
pos_list = []

for i in df['본문']:
    temp_str = ''
    try:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]+=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    except:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    pos_list.append(temp_str)

In [20]:
text

['기후정의를 말하다 기후변화 와 인종차별 은 맞닿은 문제 비즈트리뷴 마이크로소프트 측에 따르면 환경 불평등과 인종차별 간의 연관성은 세기에 일어난 환경오염 사례',
 '웹진림 소셜벤처허브 사회적기업 창업 오마이어스 지구 환경오염 기후변화 친환경 업사이클링 선한영향력',
 '지구본 입니다 불타오릅니다 환경오염은 나쁜겁니다',
 '오늘도 시원하게 환경오염',
 '환경오염',
 '탈북자들의 증언에 따르면 지금 북한에서는 프랑스 쓰레기를 수입해서 각 가정에서 이 쓰레기를 연료나 화목을 대신해서 태워 쓴다고 합니다 그래서 북한에서는 이 태운 쓰레기에서 나는 연기가 그대로 공중에 뿌려져',
 '꽊꽊 안모여 사니까 건물도 다 대체로 낮음 엘베도 많이 필요없음 아시아에 있는 공장들 주인 구미권 백인 오너 그런데 환경오염 발표하면서 동남아 수질오염 베이징 스모그 사진이런거 가져오면 이러면 쪼끔 꼴 받을 수',
 '이 커피박을 주목해야 하는 이유는 환경오염의 주범인 플라스틱만큼 중요하다 땅에 버려진 커피박이 메테인 이라는 온실가스를 뿜어내고 있기 때문이다 메테인은 이산화탄소지수 로 이산화탄소의 배에 육박하는',
 '테슬라가 돌연 결제중단 및 일론이 비트코인의 환경오염을 욕하자 비트코인은 퍼 하락장을 맞이합니다 그리고 블랙록은 바로 억원 어치의 비트코인 채굴장 주식을 사들였습니다 비트코인 시즌 끝났다고요 블랙록',
 '커피 한 잔이 남긴 커피박 그 놀라운 변화 커피박 커피박퇴비 커피찌꺼기 퇴비 환경오염 김종훈 기자',
 '커피 한 잔이 남긴 커피박 그 놀라운 변화 커피박 환경오염 퇴비 커피박퇴비 커피찌꺼기 김종훈 기자',
 '환경오염만 시키는 쓰레기',
 '잇 머스트비 뻔 잇 머스트비 쿨 잇 머스트비 쎅씨 투 마지막에 이라 한 사람이 말한건데 놀랍게도 환경오염 대처를 어떻게 해야하는가에 대한 이야기임 환경오염을 섹시하게 대처한다는게 무슨 의미냐는',
 '이 커피박을 주목해야 하는 이유는 환경오염의 주범인 플라스틱만큼 중요하다 땅에 버려진 커피박이 메테인 이라는 온실가스를 뿜어내

In [21]:
len(text)

402

In [22]:
len(twitter_df['내용'])

213